## Model wyżyłowany przeze mnie
W tym modelu nie znajdziesz typowych komentarzy szkoleniowych, a jedynie krótki opis służący nawigacji po modelu.
Jeśli masz potrzebę zrozumieć jak działa w szczególe, przejrzyj zeszyty ćwiczeń 1-3, oraz zeszyty badawcze.
To jest synteza dotychczasowych prac. Jeśli nie rozumiesz kodu pythonowskiego, to polecam StackOverflow, chociaż zawsze staram się aby był w miarę prosty i zrozumiały. Konkludując: baw się dobrze. :P

In [2]:
import mnistworkaround
import numpy as np
import tensorflow as tf
mnist = mnistworkaround.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
from tinker import RangeDict # - Jeśli chcesz używać ciągów różnych ukrytych warstw o tej samej "grubości"
                    # to zdejmij komentarz z tego importu i zmień h_lay_diff na odpowiadające Ci wartości.
                    # Oczywiście jak robisz z tego GUI, to zdejmij go permanentnie i "zamroź" do wersji maksimum.
from tinker import HLayer      # - W trakcie tworzenia automat do tworzenia
                               # kolejnych warstw za pomocą tej klasy. Na razie nie ma szans, żeby działała... ;)

In [ ]:
### PANEL GŁÓWNY ###

l_rate = 0.001          # Prędkość uczenia się modelu. Domyślna: 0.001
s_batches = 100         # Wielkość partii w jakich uczymy model (batch_size)
n_epochs = 15           # Górne organiczenie ilości "epok"
hlay_size = 50          # Domyślny rozmiar każdej ukrytej warstwy "neuronów" dla której nie postanowisz inaczej.


# Eksperymentalne zmienne dla nowego importu, nad którym pracuję w celu ułatwienia sobie i innym pracy.
# Chwilowo nic nie robi, ale mam nadzieję, że wkrótce będzie. :D
hlay_num = 2          # Definiuje ilość ukrytych warstw (Domyślnie: 2)
hlay_diff = {}        # Jeśli zdecydujesz się używać innych wartości dla różnych warstw, to wrzuć tu w formacie {1 : 30, 9 : 70} 
                        # itp. funkcja zaczerpnie stąd wartość, a jeśli jej nie zdefiniujesz, to użyje domyślnej.
                        # przy dłuższych ciągach możesz użyć "tinker.RangeDict({})", żeby ułatwić sobie życie.
                        # po prostu zmień zwyczajne nawiasy na ww funkcję i zdejmij komentarz z importu RangeDict.

# Tego nie rusz, chyba, że masz inne rozmiary obrazka grayscale, albo w ogóle zmieniłeś rodzaj inputu, np. na kolorowy RGBA:
input_size = 784        # Rozmiar danych wejściowych to 784, gdyż rozmiar obrazka w bazie danych MNIST to 28x28px=784px.
output_size = 10        # Rozmiar danych wyjściowych to 10, gdyż baza danych MNIST zawiera 10 cyfr od 0 do 9


tf.reset_default_graph() # Resetuje zmienne zapisane w pamięci modelu z jego poprzednich przebiegów.

In [ ]:
# Podstawniki dla danych wejściowych i celów.
inputs = tf.placeholder(tf.float32, [None, input_size])
targets = tf.placeholder(tf.float32, [None, output_size])

# Wagi łączące pierwszą warstwę "inputs" z 1 warstwą "ukrytą"
weights_1 = tf.get_variable("weights_1", [input_size, hlay_size])
biases_1 = tf.get_variable("biases_1", [hlay_size])
outputs_1 = tf.nn.relu(tf.matmul(inputs, weights_1) + biases_1)

# Wagi i obciążenia łączące warstwę ukrytą 1 z warstwą ukrytą 2. Do automatyzacji na panelu.
weights_2 = tf.get_variable("weights_2",[hidden_layer_size, hidden_layer_size])
biases_2 = tf.get_variable("biases_2",[hidden_layer_size])
# Zwróć uwagę z czego zwracamy wynik outputs_2, czyli wynik aktywacji drugiej warstwy.
outputs_2 = tf.nn.relu(tf.matmul(outputs_1, weights_2) + biases_2)

# Wagi łączące ostatnią warstwę ukrytą z warstwą danych wyjściowych (outputs).
weights_3 = tf.get_variable("weights_3", [hidden_layer_size, output_size])
biases_3 = tf.get_variable("biases_3", [output_size])

# Zwróć uwagę, że nie zwróciliśmy tutaj od razu funkcji aktywacyjnej, a jedynie iloczyn skalarny + obciążenie 3.

outputs = tf.matmul(outputs_2, weights_3) + biases_3

# Poniższa funkcja aplikuje zarówno Aktywator Softmax, który jest wskazany przy danych wyjściowych
# (bo daje wyskalowane przedziały ufności dzielone przez dostępne opcje sumarycznie do 1)
# oraz aplikuje koszt z funkcji logarytmicznej liczony od całkowitego błędu. 
# Format użycia to: tf.nn.softmax_cross_entropy_with_logits(logits,labels) Tutaj: logits = wyjściowe, labels = cele (targets)
loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=outputs, labels=targets)

# Poniższa metoda poprawia nam osiągi. W tej sposób TF szuka nam średnią elementów tensora w danym wymiarze.
# Więcej o zastosowaniu ENG tutaj*
mean_loss = tf.reduce_mean(loss)

# Wybieramy model treningowy. Ponieważ poszliśmy do przodu z teorią, to tym razem wybierzemy sobie ADAM,
# który jest o tyle dobry, że raczej nie zacina się na lokalnym (fałszywym) minimum funkcji i optymalnie szuka
# globalnego minimum. Jest to metoda z 2015, więc dość nowa. Daj mi znać jeśli wymyślono coś lepszego.
optimize = tf.train.AdamOptimizer(learning_rate=l_rate).minimize(mean_loss)

# Metoda tf.argmax pozwala nam wybrać która zmienna ma najwyższy "wynik" (patrz niżej) zwracany przez funkcję w sposób: 
# (outputs: z aktywatora softmax, tj. tutaj model próbuje "zgadnąć" prawidłowy wynik o który nam chodzi,
# przydzielając poszczególnym zmiennym prawdopodobieństwo. Agrmax wybiera najwyższe prawdopodobieństwo.)
# (targets: zdefiniowane przez nas w ramach bazy "labels", do czego dąży model. 
# W wypadku "OneHot" tylko jedna zmienna będzie miała wartość 1 a pozostałe 0 i to ją wybierze Argmax.)
# tf.equal (bool) porównuje te dwa ciągi, które zwracają argmax i jeśli się pokrywają zwraca 1 a jeśli nie 0.
# Otrzymujemy więc wektor zero-jedynkowy, który nabierze sensu w ramach kolejnej funkcji...
out_equals_target = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1))

# tf.reduce_mean*, o którym możesz poczytać poniżej pozwala nam logicznie obliczyć trafność modelu.
# Po prostu bierze wszystkie te 0 (nietrafione) i 1 (trafione decyzje modelu) i wyciąga średnią. Oto nasza trafność.:)
# tf.cast(obiekt_do_konwersji, docelowy_format) to nasze ubezpieczenie, na wszelki wypadek
# aby dane, które zwróci nam funkcja na pewno były w odpowiednim formacie. (tutaj tf.float32)
accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

# Odpalamy sesję i inicjalizujemy zmienne w ramach sesji. Szczegóły w zeszycie 2, pkt 5 i 6.
sess = tf.InteractiveSession()
initializer = tf.global_variables_initializer()
sess.run(initializer)

# Wielkość pakietów. Zdecyduj o wielkości pakietów na które będzie dzielona baza treningowa.
# Tzn. jak często model będzie poprawiał swoje wagi i obciążenia.
# Jeśli wpiszesz ilość równą ilości obserwacji (niezalecane)
# model przejdzie w ramach 1 epoki na raz przez całą bazę danych i będzie najdokładniejszy 
# (przykład: "Gradient Descent"), ale jest to bardzo zasobochłonne
# Im mniejszy pakiet względem bazy danych, tym częstsza aktualizacja w ramach przechodzenia przez nią,
# ale mniejsza dokładność (Jak w Stochastic Gradient Descent).
# Ogólnie warto się z tym pobawić i raczej na ogół warto poświęcić trochę dokładności na rzecz szybkości.
# Startowa : 100 mówi, że w ramach jednego "przejścia" przez bazę, 
# wagi i średnie zostaną dostosowane (ilość_przykładów / 100 razy). 
# Dzielenie "floordiv" zaokrągla ilość do równej wartości. // Edit: Wyciągnięta na panel główny.
batch_size = s_batches
# Ta zmienna to wynik dzielenia liczby przykładów treningowych w bazie danych przez rozmiar partii.
batches_number = mnist.train._num_examples // batch_size

# Poniżej dwa mechanizmy zatrzymujące:
# Ta zmienna ogranicza z góry liczbę "przejść" przez bazę, tak aby model się nie zaciął. // Edit: Wyciągnięta na panel główny.
max_epochs = n_epochs
# Ta zmienna da nam pewność, że model nie przerwie działania po pierwszym przebiegu (epoce).
# Model porównuje wyniki treningowe do bazy walidacyjnej i sprawdza, czy nie zaszło tzw. overfitting (naddopasowanie modelu)
# Wysoka wartość poprzedniej straty z walidacj przeciwdziała tutaj przerwaniu po pierwszej epoce,
# gdyż na początku model takowej wartości nie ma, więc domyślne 0 spowodowałoby niewłaściwe przerwanie, co mija się z celem...
prev_validation_loss = 9999999.

# Podobną pętlę masz w zeszycie 2. Jak coś jest niejasne sprawdź tam, bo staram się nie powtarzać.
# W tym miejscu odbywa się uczenie modelu. Zauważ, że ta pętla nie przebiegnie więcej niż 15 razy.
for epoch_counter in range(max_epochs):
    
    # Ustalamy zmienną do której pętla zapisuje obecną wartość funkcji straty.
    curr_epoch_loss = 0.
    
    # Pętla kręci się do wykorzystania wszystkich partii.
    for batch_counter in range(batches_number):
        
        # Tutaj pętla ładuje [100] wejść (input_batch) i [100] celów (target_batch) ([100] wynika z "batch_size")
        input_batch, target_batch = mnist.train.next_batch(batch_size)
        
        # Tutaj optymalizuje algorytm i kalkuluje funkcję straty dla każdej partii. 
        _, batch_loss = sess.run([optimize, mean_loss], 
            feed_dict={inputs: input_batch, targets: target_batch})
        
        # Tutaj zapisuje stratę dla obecnej iteracji pętli
        curr_epoch_loss += batch_loss
    
    # Teraz zmienna curr_epoch_loss faktycznie zawiera średnią wartość funkcji straty dla zestawu treningowego
    # dla kolejnej epoki. Otrzymujemy ją dzieląc sumę strat ze wszystkich partii w epoce przez liczbę partii.
    curr_epoch_loss /= batches_number

    # Tutaj ładujemy zestaw walidacyjny wyodrębiony już dla nas w paczce MNIST. Ten zestaw wykorzystujemy
    # wielokrotnie, i podlega on jedynie propagacji w przód, tzn. nie dostosowujemy do niego wag modelu
    # (nie trenujemy na nim). Dzieje się tak dlatego, że względem tej małej próbki (zwykle ok 10% bd)
    # sprawdzamy, czy nie zaszło naddopasowanie (overfitting).
    input_batch, target_batch = mnist.validation.next_batch(mnist.validation._num_examples)
    
    
    # Oblicza średnią stratę i trafność zestawu walidacyjnego i zapisuje je w odpowiadających im zmiennych
    validation_loss, validation_accuracy = sess.run([mean_loss, accuracy], 
                                                    feed_dict={inputs: input_batch, targets: target_batch})
    
    # Zwraca nam w widoczny sposób wartości zmiennych dla każdej 'Epoki'. Formatowanie typowo pythonowskie.
    print('Epoka: '+str(epoch_counter+1)+
          '. Strata z Treningu : '+'{0:.3f}'.format(curr_epoch_loss)+
          '. Strata z Walidacji: '+'{0:.3f}'.format(validation_loss)+
          '. Trafność Walidacji: '+'{0:.2f}'.format(validation_accuracy * 100)+'%')
    
    # Przerywa pętlę wcześniej jeśli strata z walidacji zacznie wzrastać wskazując na naddopasowanie modelu.
    if validation_loss > prev_validation_loss:
        break
        
    # Aktualizuje stratę z walidacji, żeby ją można było porównać po każdej epoce.    
    prev_validation_loss = validation_loss
        
print("Koniec Sesji Treningowej")